# cuGraph Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

### Imports

In [1]:
import cudf
import cugraph
import json 
import pandas as pd

### Create graph dataframes

We use the Star Wars social network graph prepared by Evelina Gabasova: Gabasova, E. (2016). Star Wars social network. DOI: https://doi.org/10.5281/zenodo.1411479.

In [4]:
interactions = json.loads(open('data/starwars-full-interactions.json', 'r').read())

edges = cudf.from_pandas(pd.DataFrame(interactions['links']))
nodes = cudf.from_pandas(pd.DataFrame(interactions['nodes'])).drop('colour', axis=1).reset_index()

g = cugraph.Graph()
g.from_cudf_edgelist(
    edges
    , source='source'
    , destination='target'
    , edge_attr='value'
    , renumber=True
)

---

# Graph community

---

#### cugraph.community.ecg.ecg()

In [5]:
cugraph.ecg(g, min_weight=0.05, ensemble_size=20)

,partition,vertex
0,8,62
1,9,69
2,2,73
3,2,75
4,1,77
...,...,...
104,5,92
105,6,22
106,1,85
107,1,88


#### cugraph.community.ktruss_subgraph.ktruss_subgraph()

In [14]:
cugraph.ktruss_subgraph(g, 3, use_weights=True).edges()

,src,dst
0,1,8
1,1,50
2,1,10
3,1,51
4,1,52
...,...,...
382,14,20
383,14,23
384,14,48
385,14,18


#### cugraph.community.leiden.leiden()

In [16]:
parts, modularity_score = cugraph.leiden(g, max_iter=10)

In [19]:
parts.head()

,partition,vertex
0,0,62
1,4,69
2,1,73
3,1,75
4,1,77


In [18]:
modularity_score

0.45073574781417847

#### cugraph.community.louvain.louvain()

In [21]:
parts, modularity_score = cugraph.louvain(g, max_iter=10)

In [22]:
parts.head()

,partition,vertex
0,0,62
1,4,69
2,1,73
3,1,75
4,1,77


In [23]:
modularity_score

0.45073574781417847

#### cugraph.community.subgraph_extraction.subgraph()

In [29]:
g_sub = cugraph.subgraph(g, cudf.Series([0,1,3]))

In [30]:
g_sub.nodes()

0    0
1    1
2    3
Name: 0, dtype: int64

In [31]:
g_sub.edges()

,src,dst
0,0,1
1,0,3
2,1,3


#### cugraph.community.triangle_count.triangles()

In [32]:
cugraph.triangles(g)

2013

#### cugraph.community.spectral_clustering.spectralBalancedCutClustering()

In [34]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)

In [35]:
spectral

,cluster,vertex
0,1,62
1,3,69
2,3,73
3,1,75
4,2,77
...,...,...
104,2,92
105,2,22
106,3,85
107,3,88


#### cugraph.community.spectral_clustering.analyzeClustering_edge_cut()

In [48]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_edge_cut(g, 5, spectral['cluster'])

#### cugraph.community.spectral_clustering.analyzeClustering_modularity()

In [50]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_modularity(g, 5, spectral['cluster'])

#### cugraph.community.spectral_clustering.analyzeClustering_ratio_cut()

In [55]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_ratio_cut(g, 5, spectral['cluster'])

#### cugraph.community.spectral_clustering.spectralModularityMaximizationClustering()

In [54]:
cugraph.spectralModularityMaximizationClustering(
    g
    , num_clusters=5
    , num_eigen_vects=3
    , evs_max_iter=200
    , kmean_tolerance=1e-06
    , kmean_max_iter=200
)

,cluster,vertex
0,4,62
1,0,69
2,4,73
3,4,75
4,4,77
...,...,...
104,0,92
105,3,22
106,4,85
107,4,88
